In [38]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
import os
os.chdir("E:\Kaggle_house-prices")
import tensorflow as tf
from sklearn.preprocessing import StandardScaler  
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [39]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

Step 1: tansfer objective features to numeric features

In [40]:
train['MasVnrType']=train['MasVnrType'].map({'None':0,'BrkCmn':1,'BrkFace':2,'CBlock':3,'Stone':4})
train['MasVnrType'].fillna(0,inplace=True)
test['MasVnrType']=test['MasVnrType'].map({'None':0,'BrkCmn':1,'BrkFace':2,'CBlock':3,'Stone':4})
test['MasVnrType'].fillna(0,inplace=True)

In [41]:
for features in ['BsmtQual','BsmtCond','BsmtExposure',
                 'FireplaceQu','GarageQual','GarageCond','PoolQC']:
    train[features]=train[features].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
    train[features].fillna(0,inplace=True)
    test[features]=test[features].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
    test[features].fillna(0,inplace=True)    

In [42]:
train['KitchenQual'] = train['KitchenQual'].map({'Ex': 5, 'Gd': 4,'TA':3, 'Fa':2, 'Po':1})
train['Utilities']=train['Utilities'].map({'AllPub':4,'NoSewr':3,'NoSeWa':2,'ELO':1})
train['Functional']=train['Functional'].map({'Typ': 8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4, 'Maj2':3, 'Sev':2, "Sal":1})
train['CentralAir']=train['CentralAir'].map({'Y':1,'N':0})
train['HeatingQC']=train['HeatingQC'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
train['PavedDrive']=train['PavedDrive'].map({'Y':2, 'P':1, "N":0})
train['ExterQual']=train['ExterQual'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
train['ExterCond']=train['ExterCond'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
test['KitchenQual'] = test['KitchenQual'].map({'Ex': 5, 'Gd': 4,'TA':3, 'Fa':2, 'Po':1})
test['Utilities']=test['Utilities'].map({'AllPub':4,'NoSewr':3,'NoSeWa':2,'ELO':1})
test['Functional']=test['Functional'].map({'Typ': 8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4, 'Maj2':3, 'Sev':2, "Sal":1})
test['CentralAir']=test['CentralAir'].map({'Y':1,'N':0})
test['HeatingQC']=test['HeatingQC'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
test['PavedDrive']=test['PavedDrive'].map({'Y':2, 'P':1, "N":0})
test['ExterQual']=test['ExterQual'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
test['ExterCond']=test['ExterCond'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1})
train['Electrical']=train['Electrical'].map({'SBrkr':4,'FuseA':3,'FuseF':2,'FuseP':1,'Mix':0})
train['Electrical'].fillna(train['Electrical'].median(),inplace=True)
test['Electrical']=test['Electrical'].map({'SBrkr':4,'FuseA':3,'FuseF':2,'FuseP':1,'Mix':0})
test['Electrical'].fillna(test['Electrical'].median(),inplace=True)
for name in train.columns[train.dtypes==object]:
    unique=train[name].unique()
    transform={}
    for i in range(len(unique)):
        transform[unique[i]]=i
    train[name]=train[name].map(transform)
    test[name]=test[name].map(transform)

In [43]:
train['LotFrontage'].fillna(train['LotFrontage'].mean(),inplace=True)
test['LotFrontage'].fillna(test['LotFrontage'].mean(),inplace=True)
train['MasVnrArea'].fillna(train['MasVnrArea'].median(),inplace=True)
test['MasVnrArea'].fillna(test['MasVnrArea'].median(),inplace=True)

Step2: encode Year according to range of year

In [44]:
def Year(year):
    if year<=1900:
        return 0
    elif 1900<year and year<=1920:
        return 1
    elif 1920<year and year<=1940:
        return 2
    elif 1940<year and year<=1960:
        return 3   
    elif 1960<year and year<=1980:
        return 4
    elif 1980<year and year<=2000:
        return 5
    elif 2000<year and year<=2020:
        return 6

In [45]:
train["YearRemodAdd"]=train["YearRemodAdd"].apply(Year)
train['YearBuilt']=train['YearBuilt'].apply(Year)
train['GarageYrBlt']=train['GarageYrBlt'].apply(Year)
train['GarageYrBlt'].fillna(train['YearBuilt'],inplace=True)
test["YearRemodAdd"]=test["YearRemodAdd"].apply(Year)
test['YearBuilt']=test['YearBuilt'].apply(Year)
test['GarageYrBlt']=test['GarageYrBlt'].apply(Year)
test['GarageYrBlt'].fillna(test['YearBuilt'],inplace=True)

In [46]:
chosenfeature=train.corr()[abs(train.corr().loc['SalePrice',:])>0.4].index

In [47]:
chosenfeature

Index(['OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'Foundation', 'BsmtQual', 'TotalBsmtSF', 'HeatingQC',
       '1stFlrSF', 'GrLivArea', 'FullBath', 'KitchenQual', 'TotRmsAbvGrd',
       'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'SalePrice'],
      dtype='object')

In [48]:
Train_chosen=train.loc[:,chosenfeature]

In [49]:
test_X=test.loc[:,chosenfeature[:-1]]
nacolumns=test_X.columns[test_X.isna().sum()>0]
for name in nacolumns:
    test_X[name].fillna(test_X[name].mean(),inplace=True)

In [95]:
pred_y=np.zeros((test_X.shape[0],3))
rmse_list=np.array([float('inf')]*3)

In [51]:
scaler = StandardScaler()   
scaler.fit(Train_chosen.iloc[:,:-1].values) 
Train_chosen.iloc[:,:-1]=scaler.transform(Train_chosen.iloc[:,:-1])
test_X = scaler.transform(test_X) 

Neural Network Model

In [52]:
min_rmse=float("inf")
for i in range(5):
    Train_chosen = Train_chosen.sample(frac=1)
    train_size=int(Train_chosen.shape[0]*0.7)
    train_x=Train_chosen.iloc[:train_size,:-1].values
    train_y=Train_chosen.iloc[:train_size,-1].values
    valid_x=Train_chosen.iloc[train_size:,:-1].values
    valid_y=Train_chosen.iloc[train_size:,-1].values
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(200,input_shape=(len(chosenfeature)-1,),activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(200,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
    model.fit(train_x,train_y,epochs=1000,verbose=0)
    y_pred=model.predict(valid_x).flatten()
    rmse=math.sqrt(np.sum((y_pred-valid_y)**2)/y_pred.size)
    if rmse<min_rmse:
        min_rmse=rmse
        model.save("best_nn_model")

Epoch 1/1000
1021/1021 [==============================] - 0s 458us/step - loss: 38238276540.8031 - mean_squared_error: 38238276540.8031
Epoch 2/1000
1021/1021 [==============================] - 0s 42us/step - loss: 38217749831.9608 - mean_squared_error: 38217749831.9608
Epoch 3/1000
1021/1021 [==============================] - 0s 46us/step - loss: 38138948749.4143 - mean_squared_error: 38138948749.4143
Epoch 4/1000
1021/1021 [==============================] - 0s 63us/step - loss: 37935169606.2057 - mean_squared_error: 37935169606.2057
Epoch 5/1000
1021/1021 [==============================] - 0s 47us/step - loss: 37523923589.8923 - mean_squared_error: 37523923589.8923
Epoch 6/1000
1021/1021 [==============================] - 0s 48us/step - loss: 36824295391.9060 - mean_squared_error: 36824295391.9060
Epoch 7/1000
1021/1021 [==============================] - 0s 57us/step - loss: 35784039794.0842 - mean_squared_error: 35784039794.0842
Epoch 8/1000
1021/1021 [==============================

1021/1021 [==============================] - 0s 68us/step - loss: 3980815046.3310 - mean_squared_error: 3980815046.3310
Epoch 62/1000
1021/1021 [==============================] - 0s 66us/step - loss: 3915604251.3301 - mean_squared_error: 3915604251.3301
Epoch 63/1000
1021/1021 [==============================] - 0s 70us/step - loss: 3853005676.8188 - mean_squared_error: 3853005676.8188
Epoch 64/1000
1021/1021 [==============================] - 0s 90us/step - loss: 3792374065.3947 - mean_squared_error: 3792374065.3947
Epoch 65/1000
1021/1021 [==============================] - 0s 54us/step - loss: 3731956753.5514 - mean_squared_error: 3731956753.5514
Epoch 66/1000
1021/1021 [==============================] - 0s 73us/step - loss: 3665802333.5240 - mean_squared_error: 3665802333.5240
Epoch 67/1000
1021/1021 [==============================] - 0s 67us/step - loss: 3606500974.0725 - mean_squared_error: 3606500974.0725
Epoch 68/1000
1021/1021 [==============================] - 0s 35us/step - lo

1021/1021 [==============================] - 0s 50us/step - loss: 1800250057.8413 - mean_squared_error: 1800250057.8413
Epoch 123/1000
1021/1021 [==============================] - 0s 46us/step - loss: 1787809103.3575 - mean_squared_error: 1787809103.3575
Epoch 124/1000
1021/1021 [==============================] - 0s 48us/step - loss: 1773981012.6856 - mean_squared_error: 1773981012.6856
Epoch 125/1000
1021/1021 [==============================] - 0s 49us/step - loss: 1756669789.7747 - mean_squared_error: 1756669789.7747
Epoch 126/1000
1021/1021 [==============================] - 0s 44us/step - loss: 1747463853.3203 - mean_squared_error: 1747463853.3203
Epoch 127/1000
1021/1021 [==============================] - 0s 45us/step - loss: 1731561534.8090 - mean_squared_error: 1731561534.8090
Epoch 128/1000
1021/1021 [==============================] - 0s 49us/step - loss: 1721675839.8119 - mean_squared_error: 1721675839.8119
Epoch 129/1000
1021/1021 [==============================] - 0s 44us/st

1021/1021 [==============================] - 0s 44us/step - loss: 1326178927.0127 - mean_squared_error: 1326178927.0127
Epoch 183/1000
1021/1021 [==============================] - 0s 51us/step - loss: 1321531458.5700 - mean_squared_error: 1321531458.5700
Epoch 184/1000
1021/1021 [==============================] - 0s 47us/step - loss: 1320270163.6200 - mean_squared_error: 1320270163.6200
Epoch 185/1000
1021/1021 [==============================] - 0s 46us/step - loss: 1325658782.9657 - mean_squared_error: 1325658782.9657
Epoch 186/1000
1021/1021 [==============================] - 0s 59us/step - loss: 1315991845.3595 - mean_squared_error: 1315991845.3595
Epoch 187/1000
1021/1021 [==============================] - 0s 47us/step - loss: 1307759216.2664 - mean_squared_error: 1307759216.2664
Epoch 188/1000
1021/1021 [==============================] - 0s 45us/step - loss: 1302824797.0225 - mean_squared_error: 1302824797.0225
Epoch 189/1000
1021/1021 [==============================] - 0s 69us/st

1021/1021 [==============================] - 0s 55us/step - loss: 1183311474.2723 - mean_squared_error: 1183311474.2723
Epoch 243/1000
1021/1021 [==============================] - 0s 48us/step - loss: 1178477442.8834 - mean_squared_error: 1178477442.8834
Epoch 244/1000
1021/1021 [==============================] - 0s 44us/step - loss: 1182127966.2135 - mean_squared_error: 1182127966.2135
Epoch 245/1000
1021/1021 [==============================] - 0s 52us/step - loss: 1178532282.7973 - mean_squared_error: 1178532282.7973
Epoch 246/1000
1021/1021 [==============================] - 0s 44us/step - loss: 1175643455.4358 - mean_squared_error: 1175643455.4358
Epoch 247/1000
1021/1021 [==============================] - 0s 44us/step - loss: 1177520416.5328 - mean_squared_error: 1177520416.5328
Epoch 248/1000
1021/1021 [==============================] - 0s 52us/step - loss: 1171612062.0255 - mean_squared_error: 1171612062.0255
Epoch 249/1000
1021/1021 [==============================] - 0s 45us/st

...

1021/1021 [==============================] - 0s 53us/step - loss: 826182397.8061 - mean_squared_error: 826182397.8061
Epoch 713/1000
1021/1021 [==============================] - 0s 49us/step - loss: 825877232.3291 - mean_squared_error: 825877232.3291
Epoch 714/1000
1021/1021 [==============================] - 0s 50us/step - loss: 827316360.2429 - mean_squared_error: 827316360.2429
Epoch 715/1000
1021/1021 [==============================] - 0s 55us/step - loss: 827168960.3134 - mean_squared_error: 827168960.3134
Epoch 716/1000
1021/1021 [==============================] - 0s 48us/step - loss: 827252471.0989 - mean_squared_error: 827252471.0989
Epoch 717/1000
1021/1021 [==============================] - 0s 50us/step - loss: 824469648.3918 - mean_squared_error: 824469648.3918
Epoch 718/1000
1021/1021 [==============================] - 0s 57us/step - loss: 826835021.5397 - mean_squared_error: 826835021.5397
Epoch 719/1000
1021/1021 [==============================] - 0s 48us/step - loss: 826

1021/1021 [==============================] - 0s 48us/step - loss: 818610392.0705 - mean_squared_error: 818610392.0705
Epoch 774/1000
1021/1021 [==============================] - 0s 53us/step - loss: 818633035.4711 - mean_squared_error: 818633035.4711
Epoch 775/1000
1021/1021 [==============================] - 0s 49us/step - loss: 819450679.7884 - mean_squared_error: 819450679.7884
Epoch 776/1000
1021/1021 [==============================] - 0s 53us/step - loss: 818923201.7551 - mean_squared_error: 818923201.7551
Epoch 777/1000
1021/1021 [==============================] - 0s 48us/step - loss: 820049345.5671 - mean_squared_error: 820049345.5671
Epoch 778/1000
1021/1021 [==============================] - 0s 51us/step - loss: 815144670.6523 - mean_squared_error: 815144670.6523
Epoch 779/1000
1021/1021 [==============================] - 0s 57us/step - loss: 818929745.9902 - mean_squared_error: 818929745.9902
Epoch 780/1000
1021/1021 [==============================] - 0s 48us/step - loss: 819

1021/1021 [==============================] - 0s 49us/step - loss: 809555229.4613 - mean_squared_error: 809555229.4613
Epoch 835/1000
1021/1021 [==============================] - 0s 52us/step - loss: 812329115.2047 - mean_squared_error: 812329115.2047
Epoch 836/1000
1021/1021 [==============================] - 0s 48us/step - loss: 811979104.5955 - mean_squared_error: 811979104.5955
Epoch 837/1000
1021/1021 [==============================] - 0s 56us/step - loss: 807241135.4515 - mean_squared_error: 807241135.4515
Epoch 838/1000
1021/1021 [==============================] - 0s 48us/step - loss: 808609133.2576 - mean_squared_error: 808609133.2576
Epoch 839/1000
1021/1021 [==============================] - 0s 49us/step - loss: 809267645.4927 - mean_squared_error: 809267645.4927
Epoch 840/1000
1021/1021 [==============================] - 0s 55us/step - loss: 809019434.8129 - mean_squared_error: 809019434.8129
Epoch 841/1000
1021/1021 [==============================] - 0s 48us/step - loss: 807

1021/1021 [==============================] - 0s 47us/step - loss: 798636298.8443 - mean_squared_error: 798636298.8443
Epoch 896/1000
1021/1021 [==============================] - 0s 56us/step - loss: 800034093.7591 - mean_squared_error: 800034093.7591
Epoch 897/1000
1021/1021 [==============================] - 0s 49us/step - loss: 799077443.0715 - mean_squared_error: 799077443.0715
Epoch 898/1000
1021/1021 [==============================] - 0s 50us/step - loss: 797992184.1332 - mean_squared_error: 797992184.1332
Epoch 899/1000
1021/1021 [==============================] - 0s 60us/step - loss: 798889647.6396 - mean_squared_error: 798889647.6396
Epoch 900/1000
1021/1021 [==============================] - 0s 47us/step - loss: 800388167.1146 - mean_squared_error: 800388167.1146
Epoch 901/1000
1021/1021 [==============================] - 0s 53us/step - loss: 799720053.7199 - mean_squared_error: 799720053.7199
Epoch 902/1000
1021/1021 [==============================] - 0s 48us/step - loss: 798

1021/1021 [==============================] - 0s 52us/step - loss: 789483529.0578 - mean_squared_error: 789483529.0578
Epoch 957/1000
1021/1021 [==============================] - 0s 50us/step - loss: 792684722.1469 - mean_squared_error: 792684722.1469
Epoch 958/1000
1021/1021 [==============================] - 0s 47us/step - loss: 791177030.4564 - mean_squared_error: 791177030.4564
Epoch 959/1000
1021/1021 [==============================] - 0s 53us/step - loss: 793393133.8217 - mean_squared_error: 793393133.8217
Epoch 960/1000
1021/1021 [==============================] - 0s 50us/step - loss: 792325314.6954 - mean_squared_error: 792325314.6954
Epoch 961/1000
1021/1021 [==============================] - 0s 48us/step - loss: 790269997.4456 - mean_squared_error: 790269997.4456
Epoch 962/1000
1021/1021 [==============================] - 0s 68us/step - loss: 789781403.6748 - mean_squared_error: 789781403.6748
Epoch 963/1000
1021/1021 [==============================] - 0s 48us/step - loss: 790

In [53]:
print("The min rmse of NN is %.0f."%(min_rmse))

The min rmse of NN is 29396.


In [54]:
best_nn_model=keras.models.load_model("best_nn_model")
pred_y_nn=best_nn_model.predict(test_X).flatten()

In [96]:
if min_rmse<rmse_list.max():
    position=rmse_list.argmax()
    rmse_list[position]=min_rmse
    pred_y[:,position]=pred_y_nn

In [97]:
pred_y_nn

array([123451.945, 141713.75 , 157197.25 , ..., 159981.66 , 122810.4  ,
       231253.97 ], dtype=float32)

KNN

In [85]:
min_rmse=float("inf")
kf=KFold(n_splits=5,shuffle=False)
kf.split(Train_chosen)
for i in range(5):
    parameters={'n_neighbors':5*i+10, 'weights':'distance'}
    knnr = KNeighborsRegressor(**parameters)
    Train_chosen = Train_chosen.sample(frac=1)
    rmse=0
    for train_index,test_index in kf.split(Train_chosen):
        train_x,valid_x=Train_chosen.iloc[train_index,:-1],Train_chosen.iloc[test_index,:-1]
        train_y,valid_y=Train_chosen.iloc[train_index,-1],Train_chosen.iloc[test_index,-1]            
        knnr.fit(train_x, train_y)
        y_pred=knnr.predict(valid_x).flatten()
        rmse+=math.sqrt(np.sum((y_pred-valid_y)**2)/y_pred.size)/5
    if rmse<min_rmse:
        min_rmse=rmse
        best_p=parameters

In [86]:
print("Min RMSE is %0f, best parameters is: "%(min_rmse), best_p)

Min RMSE is 34778.992653, best parameters is:  {'n_neighbors': 15, 'weights': 'distance'}


In [58]:
knnr = KNeighborsRegressor(**best_p)
trainX=Train_chosen.iloc[:,:-1].values
trainY=Train_chosen.iloc[:,-1].values
knnr.fit(trainX, trainY)
pred_y_knn=knnr.predict(test_X)

In [98]:
if min_rmse<rmse_list.max():
    position=rmse_list.argmax()
    rmse_list[position]=min_rmse
    pred_y[:,position]=pred_y_knn

In [99]:
pred_y_knn

array([127835.40339177, 143967.05245296, 175691.94168543, ...,
       161209.86903644, 109045.84912474, 205174.7863266 ])

Random Forest

In [81]:
min_rmse=float("inf")
Train_chosen = Train_chosen.sample(frac=1)
kf=KFold(n_splits=5,shuffle=False)
kf.split(Train_chosen)
for i in range(5):
    parameters={'n_estimators':10*i+10}
    RF =RandomForestRegressor(**parameters)
    rmse=0
    for train_index,test_index in kf.split(Train_chosen):
        train_x,valid_x=Train_chosen.iloc[train_index,:-1],Train_chosen.iloc[test_index,:-1]
        train_y,valid_y=Train_chosen.iloc[train_index,-1],Train_chosen.iloc[test_index,-1]            
        RF.fit(trainX, trainY)
        y_pred=RF.predict(valid_x).flatten()
        rmse+=math.sqrt(np.sum((y_pred-valid_y)**2)/y_pred.size)/5
    if rmse<min_rmse:
        min_rmse=rmse
        best_p=parameters

In [82]:
print("Min RMSE is %0f, best parameters is: "%(min_rmse), best_p)

Min RMSE is 12013.968016, best parameters is:  {'n_estimators': 40}


In [83]:
RF =RandomForestRegressor(**best_p)
trainX=Train_chosen.iloc[:,:-1].values
trainY=Train_chosen.iloc[:,-1].values
RF.fit(trainX, trainY)
pred_y_RF=RF.predict(test_X)

In [100]:
if min_rmse<rmse_list.max():
    position=rmse_list.argmax()
    rmse_list[position]=min_rmse
    pred_y[:,position]=pred_y_RF

In [101]:
pred_y_RF

array([129225.  , 146200.85, 172269.5 , ..., 154379.35, 105633.75,
       217474.45])

Lasso Regression

In [87]:
train_x,valid_x=Train_chosen.iloc[train_index,:-1],Train_chosen.iloc[test_index,:-1]
train_y,valid_y=Train_chosen.iloc[train_index,-1],Train_chosen.iloc[test_index,-1]            
RF.fit(trainX, trainY)
y_pred=RF.predict(valid_x).flatten()
rmse=math.sqrt(np.sum((y_pred-valid_y)**2)/y_pred.size)
rmse

10966.75255932406

In [88]:
model_lasso=Lasso(alpha=0.01)
model_lasso.fit(trainX,trainY)
pred_y_LS=model_lasso.predict(test_X)

In [102]:
if rmse<rmse_list.max():
    position=rmse_list.argmax()
    rmse_list[position]=rmse
    pred_y[:,position]=pred_y_LS

In [103]:
pred_y_LS

array([112084.13424218, 153490.27438797, 161084.75950146, ...,
       153532.20425981, 103026.06789066, 218432.46199892])

XGB

In [111]:
min_rmse=float('inf')
parameters=[]
for col in [0.3,0.5,0.7,0.9]:
    for depth in [10,15,20]:
        for n in [30,40,50]:
            parameters.append([col,depth,n])
for p in parameters:
    params = {"objective":"reg:linear",'colsample_bytree': 0.5,'learning_rate': 0.1,
                'max_depth': 30, 'alpha': 10,'n_estimators':40}
    params['colsample_bytree']=p[0]
    params['max_depty']=p[1]
    params['n_estimators']=p[2]
    data_dmatrix = xgb.DMatrix(data=trainX,label=trainY)
    cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", seed=123)
    test_rmse=cv_results.loc[49,'test-rmse-mean']
    if test_rmse<min_rmse:
        best_params=params
        min_rmse=test_rmse

[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=7
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=7
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=6
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_

[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 316 extra nodes, 0 pruned nodes, max_depth=16
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 288 extra nodes, 0 pruned nodes, max_depth=15
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 240 extra nodes, 0 pruned nodes, max_depth=16
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 0 pruned nodes, max_depth=13
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=13
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 292 extra nodes, 0 pruned nodes, max_depth=17
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=12
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 306 extra nodes, 0 pruned nodes, max_depth=17
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 260 extra nodes, 0 p

[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1082 extra nodes, 0 pruned nodes, max_depth=25
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1100 extra nodes, 0 pruned nodes, max_depth=25
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 320 extra nodes, 0 pruned nodes, max_depth=15
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 508 extra nodes, 0 pruned nodes, max_depth=19
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1134 extra nodes, 0 pruned nodes, max_depth=21
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 470 extra nodes, 0 pruned nodes, max_depth=14
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 550 extra nodes, 0 pruned nodes, max_depth=21
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1188 extra nodes, 0 pruned nodes, max_depth=26
[14:13:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1156 extra nodes

[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 412 extra nodes, 0 pruned nodes, max_depth=15
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1372 extra nodes, 0 pruned nodes, max_depth=30
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=5
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=7
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes,

[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=11
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 274 extra nodes, 0 pruned nodes, max_depth=14
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 316 extra nodes, 0 pruned nodes, max_depth=16
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 288 extra nodes, 0 pruned nodes, max_depth=15
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 240 extra nodes, 0 pruned nodes, max_depth=16
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 0 pruned nodes, max_depth=13
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=13
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 292 extra nodes, 0 pruned nodes, max_depth=17
[14:13:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 p

...

[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 186 extra nodes, 0 pruned nodes, max_depth=11
[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 224 extra nodes, 0 pruned nodes, max_depth=12
[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 214 extra nodes, 0 pruned nodes, max_depth=16
[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 204 extra nodes, 0 pruned nodes, max_depth=13
[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 0 pruned nodes, max_depth=14
[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 262 extra nodes, 0 pruned nodes, max_depth=15
[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 258 extra nodes, 0 pruned nodes, max_depth=15
[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 266 extra nodes, 0 pruned nodes, max_depth=12
[14:14:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 274 extra nodes, 0 p

[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 834 extra nodes, 0 pruned nodes, max_depth=22
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 806 extra nodes, 0 pruned nodes, max_depth=25
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 770 extra nodes, 0 pruned nodes, max_depth=24
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 876 extra nodes, 0 pruned nodes, max_depth=20
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 848 extra nodes, 0 pruned nodes, max_depth=25
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 828 extra nodes, 0 pruned nodes, max_depth=23
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 826 extra nodes, 0 pruned nodes, max_depth=27
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 822 extra nodes, 0 pruned nodes, max_depth=22
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 816 extra nodes, 0 p

[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1078 extra nodes, 0 pruned nodes, max_depth=29
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1074 extra nodes, 0 pruned nodes, max_depth=27
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1046 extra nodes, 0 pruned nodes, max_depth=30
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=6
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=7
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=7
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=6
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=8
[14:14:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned no

[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 444 extra nodes, 0 pruned nodes, max_depth=22
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 364 extra nodes, 0 pruned nodes, max_depth=15
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 400 extra nodes, 0 pruned nodes, max_depth=16
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 442 extra nodes, 0 pruned nodes, max_depth=17
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 388 extra nodes, 0 pruned nodes, max_depth=18
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 454 extra nodes, 0 pruned nodes, max_depth=14
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 386 extra nodes, 0 pruned nodes, max_depth=18
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 456 extra nodes, 0 pruned nodes, max_depth=17
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 460 extra nodes, 0 p

[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 872 extra nodes, 0 pruned nodes, max_depth=23
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 868 extra nodes, 0 pruned nodes, max_depth=27
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 878 extra nodes, 0 pruned nodes, max_depth=23
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 910 extra nodes, 0 pruned nodes, max_depth=25
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 924 extra nodes, 0 pruned nodes, max_depth=22
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 848 extra nodes, 0 pruned nodes, max_depth=26
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 916 extra nodes, 0 pruned nodes, max_depth=22
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 890 extra nodes, 0 pruned nodes, max_depth=25
[14:14:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 852 extra nodes, 0 p

In [92]:
print("Min RMSD is %0f, best parameters is: "%(min_rmse), best_params)

Min RMSD is 30644.014844, best parameters is:  {'objective': 'reg:linear', 'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 30, 'alpha': 10, 'n_estimators': 30, 'max_depty': 10}


In [93]:
xg_reg = xgb.XGBRegressor(**best_params)
trainX=Train_chosen.iloc[:,:-1].values
trainY=Train_chosen.iloc[:,-1].values
xg_reg.fit(trainX, trainY)
pred_y_xgb=xg_reg.predict(test_X)

In [104]:
if min_rmse<rmse_list.max():
    position=rmse_list.argmax()
    rmse_list[position]=min_rmse
    pred_y[:,position]=pred_y_xgb

Combine all the prediction values and use the mean as the final prediction

In [105]:
test['SalePrice']=pred_y.mean(axis=1)

In [107]:
submission=test.loc[:,["Id",'SalePrice']]

In [108]:
submission.to_csv("Submission.csv",index=False)